<a href="https://colab.research.google.com/github/srivatsav29/SmartResumeSort/blob/main/SmartResumeSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1: Install Required Libraries
!pip install transformers pandas pdfplumber --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 73.0 MB/s eta 0:00:00


In [2]:
# STEP 2: Import Libraries
import pandas as pd
import pdfplumber
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import os
from google.colab import files

In [3]:
# STEP 3: Load a Pretrained Model (FLAN-T5)
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
# STEP 4: Define Classifier Function
categories = ["Data Science", "Human Resources", "Software Development"]

def classify_resume(text):
    prompt = f"""
    Classify this resume into one of the following categories:
    {', '.join(categories)}

    Resume:
    {text}

    Category:
    """
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=20)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction.strip()

In [5]:
# STEP 5: Upload Resume Files or a CSV
print("Please upload a CSV file with a column named 'resume_text' or PDF resumes")
uploaded = files.upload()

pdf_results = []
csv_loaded = False

for filename in uploaded.keys():
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)
        csv_loaded = True
    elif filename.endswith(".pdf"):
        with pdfplumber.open(filename) as pdf:
            full_text = " ".join([page.extract_text() or "" for page in pdf.pages])
        predicted = classify_resume(full_text)
        pdf_results.append({"filename": filename, "category": predicted})

Please upload a CSV file with a column named 'resume_text' or PDF resumes


Saving Karthik_Nandeti_Resume.pdf to Karthik_Nandeti_Resume.pdf


In [6]:
# STEP 6: Process Results
if csv_loaded:
    df['Predicted Category'] = df['resume_text'].apply(classify_resume)
    df.to_csv("classified_resumes.csv", index=False)
    files.download("classified_resumes.csv")
    print("CSV classification complete. File downloaded.")

if pdf_results:
    df_pdf = pd.DataFrame(pdf_results)
    df_pdf.to_csv("pdf_resume_classification.csv", index=False)
    files.download("pdf_resume_classification.csv")
    print("PDF resume classification complete. File downloaded.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF resume classification complete. File downloaded.
